In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)


In [2]:
from quaintscience.trader.service.backtester import BackTesterService

In [3]:
scrip = "NIFTY 50"
exchange = "NSE"

In [19]:
service = BackTesterService(from_date="20230101", to_date="20231231", interval="10min",
                          redis_server="localhost", redis_port=6379, instruments=f"{scrip}:{exchange}",
                            cache_path="../data_cache/kite")

LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20230306-20230505.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20230704-20230902.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20230902-20231101.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20230105-20230306.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20230505-20230704.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20231101-20231106.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20230907-20231106.csv
LOG | info: Reading ../data_cache/kite/historical_data/NSE/NIFTY_50/data-NIFTY_50-NSE-20221106-20230105.csv
LOG | info: Read 9424 rows.


In [ ]:
instrument = {"scrip": scrip, "exchange": exchange}
df = service.trade_manager.get_historic_data(scrip=instrument["scrip"],
                                                      exchange=instrument["exchange"],
                                                      interval=service.interval,
                                                      from_date=service.from_date,
                                                      to_date=service.to_date,
                                                      download=False)

In [7]:
from quaintscience.trader.core.indicator import HeikinAshiIndicator
#df = HeikinAshiIndicator().compute(df)[0]

In [8]:
#df = donchain_indicator.compute(df)[0]

In [9]:
from quaintscience.trader.core.indicator import WMAIndicator

In [10]:
df = WMAIndicator(period=20).compute(df)[0]
df = WMAIndicator(period=10).compute(df)[0]

In [11]:
from quaintscience.trader.core.indicator import DonchainPullbackLocator

In [12]:
df = DonchainPullbackLocator(period=15, data_period="10min").compute(df)[0]

Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)
Found pullback (upper)


In [13]:
factor = 0.04/100.
df["WMA20Upper"] = df["WMA20"] * (1 + factor)
df["WMA20Lower"] = df["WMA20"] * (1 - factor)

In [14]:
from quaintscience.trader.core.graphing import backtesting_results_plot
import numpy as np

In [15]:
from quaintscience.trader.core.indicator import BreakoutIndicator

In [16]:
df = BreakoutIndicator(upper_breakout_column="donchainUpper",
                       lower_breakout_column="donchainLower",
                       data_interval="10min").compute(df)[0]

In [17]:
backtesting_results_plot(df, [], indicator_fields=[#{"field": "lowerPullback", "panel": 1},
                                                   #{"field": "upperPullback", "panel": 1},
                                                    {"field": "donchainUpper_breakout", "panel": 1},
                                                    {"field": "donchainUpper_breakout", "panel": 1},
                                                   "donchainUpper",
                                                   "donchainMiddle",
                                                   "WMA20",
                                                   "WMA10",
                                                   "donchainLower"], mpf_custom_kwargs={"ylim": (np.mean(df["close"]) - 1.5 * df["close"].std(), np.mean(df["close"]) + 1.5 * df["close"].std()), "fill_between": {"y1": df["WMA20Upper"].fillna(0.).values, "y2": df["WMA20Lower"].fillna(0.).values, "alpha": 0.5}})

/code/github/finance/quaintrade/venv/lib/python3.11/site-packages/mplfinance/_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+


In [17]:
y

NameError: name 'y' is not defined

In [ ]:
(df["lowerPullback"] > 0).sum()

In [ ]:
(df["upperPullback"] > 0).sum()

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal, fftpack
import numpy as np


In [ ]:
#f, t, Sxx = signal.spectrogram(df["close"].to_numpy(), 5)
#plt.pcolormesh(t, f, Sxx, shading='gouraud')
#plt.specgram(df["close"], Fs=12, cmap="rainbow")
#plt.show()

In [ ]:
#fig, axs = plt.subplots(2)

#a1 = df[df.index < "2023-04-01"]["high"].to_numpy()
#x, y = np.histogram(a1, bins=250)
#a2 = df[df.index >= "2023-04-01"]["high"].to_numpy()
#x1, y1 = np.histogram(a2, bins=250)

In [ ]:
#axs[0].plot(y[:-1], x)
#axs[1].plot(y1[:-1], x1)
#plt.show()


In [ ]:
#print(len(df[df.index < "2023-04-01"]["close"]), len(df[df.index >= "2023-04-01"]["close"]))